In [ ]:
from keras import losses, models, optimizers
from keras.models import Sequential
from keras.layers import (Dense, Dropout, Activation, Flatten) 
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.datasets import load_boston 
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet, Lasso, Ridge
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from geopy import distance
import tensorflow as tf

In [ ]:
data_n = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/train.csv', nrows = 7000, parse_dates=["pickup_datetime"])
data_n.shape, data_n.shape

In [ ]:
data_n

In [ ]:
data_n.isnull().sum().sort_index()/len(data_n)

Ввыборке незначительное количество нулевых записей, поэтому их можно будет удалить из выборки. Теперь можно просмотреть статистику обучающей выборки и тестовой выборки, чтобы найти аномалии.

In [ ]:
data_n.describe()

В наборе данных есть выбросы. Включение этих точек данных в обучающий набор не позволит нашей модели работать эффективно, поэтому лучше с ними справиться. Стоимость проезда варьируется от -62 до почти 1300 долларов, что не имеет смысла. Базовая плата за проезд в такси Нью-Йорка составляет 2,50 доллара США, поэтому ограничиваю стоимость проезда от 2,50 долларов США до 500 долларов США. Количество пассажиров увеличивается до 208, что не имеет смысла, поэтому ограничиваю его максимальной вместимостью, которая составляет 5 пассажиров.

In [ ]:
data_n.drop(data_n[data_n['pickup_longitude'] == 0].index, axis=0, inplace = True)
data_n.drop(data_n[data_n['pickup_latitude'] == 0].index, axis=0, inplace = True)
data_n.drop(data_n[data_n['dropoff_longitude'] == 0].index, axis=0, inplace = True)
data_n.drop(data_n[data_n['dropoff_latitude'] == 0].index, axis=0, inplace = True)
data_n.drop(data_n[data_n['passenger_count'] == 208].index, axis=0, inplace = True)
data_n.drop(data_n[data_n['passenger_count'] > 5].index, axis=0, inplace = True)
data_n.drop(data_n[data_n['passenger_count'] == 0].index, axis=0, inplace = True)

In [ ]:
data_n.dropna(inplace=True)

data_n.drop(data_n.index[(data_n.pickup_longitude < -75) | 
           (data_n.pickup_longitude > -72) | 
           (data_n.pickup_latitude < 40) | 
           (data_n.pickup_latitude > 42)],inplace=True)
data_n.drop(data_n.index[(data_n.dropoff_longitude < -75) | 
           (data_n.dropoff_longitude > -72) | 
           (data_n.dropoff_latitude < 40) | 
           (data_n.dropoff_latitude > 42)],inplace=True)
data_n = data_n.reset_index(drop=True)

Долгота города Нью-Йорка колеблется от -75 до -72. Широта находится в диапазоне от 40 до 42. В наборе данных есть несколько точек, которые лежат за этими пределами. Эти точки будут удалены, так как они находятся за пределами города.

In [ ]:
distance_miles = []

for i in range(len(data_n.pickup_latitude)):
    distance_miles.append(distance.distance((data_n.pickup_latitude[i], data_n.pickup_longitude[i]), (data_n.dropoff_latitude[i], data_n.dropoff_longitude[i])).km)

data_n['distance_miles'] = distance_miles 
data_n

In [ ]:
is_null = pd.isnull(data_n.distance_miles).sum()
print(is_null)

In [ ]:
fig=plt.figure(figsize=(11, 8))
ax1=fig.add_subplot(1,1,1)
ax1.scatter(data_n.distance_miles,data_n.fare_amount,  color='g', alpha=0.2)
ax1.set_title('The graph of distance-related charges')
ax1.set_xlabel("Distance")
ax1.set_ylabel("Payment")

fig3=plt.figure(figsize=(11, 8))
ax5=fig3.add_subplot(1,1,1)
ax5.scatter(data_n.passenger_count, data_n.fare_amount, color='green')
ax5.set_title('The graph of payment depending on the number of passengers')
ax5.set_xlabel('Number of passengers')
ax5.set_ylabel('Payment')

data_n['year'] = data_n.pickup_datetime.apply(lambda t: t.year)
data_n['weekday'] = data_n.pickup_datetime.apply(lambda t: t.weekday())
data_n['hour'] = data_n.pickup_datetime.apply(lambda t: t.hour)

fig2=plt.figure(figsize=(11, 8))
ax5=fig2.add_subplot(1,1,1)
ax5.scatter(data_n.weekday, data_n.fare_amount, color='green')
ax5.set_title('The graph of payment depending on the day of the week')
ax5.set_xlabel('Day of the week')
ax5.set_ylabel('Payment')


fig2=plt.figure(figsize=(11, 8))
ax5=fig2.add_subplot(1,1,1)
ax5.scatter(data_n.hour, data_n.fare_amount, color='green')
ax5.set_title('The graph of payment depending on the call time')
ax5.set_xlabel('Call time')
ax5.set_ylabel('Payment')

In [ ]:
lr = LinearRegression()
x = data_n.distance_miles
y = data_n.fare_amount
x = np.transpose(np.atleast_2d(x))
lr.fit(x,y)
y_pred = lr.predict(x)
mse_lin_rm = mean_squared_error(y, y_pred)
rmse_lin_rm = np.sqrt(mse_lin_rm)
r2_lin_rm = r2_score(y, y_pred) 

print("Accuracy: %0.2f (+/- %0.2f)" % (np.mean(mse_lin_rm), np.std(mse_lin_rm) * 2))
print("Mean R^2: %0.2f\n" % (np.mean(r2_lin_rm)))    

fig=plt.figure(figsize=(7, 5))
ax5=fig.add_subplot(1,1,1)
ax5.scatter(data_n.distance_miles,data_n.fare_amount, color='g', alpha=0.2, norm=0.3)
ax5.plot(x,y_pred)
ax5.set_title('Pairwise linear regression by least squares method ')
ax5.set_xlabel('Distance ')
ax5.set_ylabel('Payment $USD')

In [ ]:
# Множинна лінійна регресія
x_mul = data_n[['passenger_count','distance_miles']]
y = data_n.fare_amount

X_train, X_test, y_train, y_test = train_test_split(x_mul, y, test_size=0.2, random_state=5) 

lr_mul = LinearRegression()
lr_mul.fit(X_train, y_train)

p = lr_mul.predict(x_mul)
# Spliting

y_pred = lr_mul.predict(X_test)
mse_lin_mul = mean_squared_error(y_test, y_pred)
r2_mul = r2_score(y_pred, y_test)
 
print("\nAccuracy: %0.2f (+/- %0.2f)" % (np.mean(mse_lin_mul), np.std(mse_lin_mul) * 2))
print("Mean R^2: %0.2f\n" % (np.mean(r2_mul)))        
    
fig=plt.figure(figsize=(7, 5))
mulreg=fig.add_subplot(1,1,1)
mulreg.scatter(p, y, color='r', alpha=0.1)
mulreg.set_title('multiLinear Regression ')

> Множественной называют линейную регрессию, в модели которой число независимых переменных две или более(passenger_count', 'distance_miles, 'fare_amount').

In [ ]:
# Множинна лінійна регресія Lasso

x_mul = data_n[['passenger_count','distance_miles']]
y = data_n.fare_amount

X_train, X_test, y_train, y_test = train_test_split(x_mul, y, test_size=0.1, random_state=0)    

lr_mul = Lasso(alpha=0.1)
lr_mul.fit(X_train, y_train)
y_pred = lr_mul.predict(X_test)
mse_lin_mul = mean_squared_error(y_test, y_pred)
r2_mul = r2_score(y_pred, y_test) 

print("\nAccuracy: %0.2f (+/- %0.2f)" % (np.mean(mse_lin_mul), np.std(mse_lin_mul) * 2))
print("Mean R^2: %0.2f\n" % (np.mean(r2_mul))) 

fig=plt.figure(figsize=(7, 5))
mulreg=fig.add_subplot(1,1,1)
mulreg.scatter(p, y, color='g', alpha=0.1)
mulreg.set_title('Множественная линейная регрессия (Lasso)')

> Метод регрессии “лассо” (LASSO, Least Absolute Shrinkage and Selection Operator) заключается во введении дополнительного слагаемого регуляризации в функционал оптимизации модели, что часто позволяет получать более устойчивое решение**

In [ ]:
# Множинна лінійна регресія Ridge

x_mul = data_n[['passenger_count','distance_miles']]
y = data_n.fare_amount

X_train, X_test, y_train, y_test = train_test_split(x_mul, y, test_size=0.24, random_state=0)    

lr_mul = Ridge(alpha=0.1)
lr_mul.fit(X_train, y_train)
y_pred = lr_mul.predict(X_test)
mse_lin_mul = mean_squared_error(y_test, y_pred)
r2_mul = r2_score(y_pred, y_test) 

print("\nAccuracy: %0.2f (+/- %0.2f)" % (np.mean(mse_lin_mul), np.std(mse_lin_mul) * 2))
print("Mean R^2: %0.2f\n" % (np.mean(r2_mul))) 

fig=plt.figure(figsize=(7, 5))
mulreg=fig.add_subplot(1,1,1)
mulreg.scatter(p, y, color='g', alpha=0.1)
mulreg.set_title('Множественная линейная регрессия (Ridge)')

> Ридж-регрессия или гребневая регрессия (англ. ridge regression) - это один из методов понижения размерности. Часто его применяют для борьбы с переизбыточностью данных, когда независимые переменные коррелируют друг с другом (т.е. имеет место мультиколлинеарность).

In [ ]:
# Множинна лінійна регресія ElasticNet

x_mul = data_n[['passenger_count','distance_miles']]
y = data_n.fare_amount

X_train, X_test, y_train, y_test = train_test_split(x_mul, y, test_size=0.2, random_state=0)    

lr_mul = ElasticNet(alpha=0.1)
lr_mul.fit(X_train, y_train)
y_pred = lr_mul.predict(X_test)
mse_lin_mul_l = mean_squared_error(y_test, y_pred)
r2_mul = r2_score(y_pred, y_test) 
    
print("\nAccuracy: %0.2f (+/- %0.2f)" % (np.mean(mse_lin_mul_l), np.std(mse_lin_mul_l) * 2))
print("Mean R^2: %0.2f\n" % (np.mean(r2_mul))) 
    
fig=plt.figure(figsize=(7, 5))
mulreg=fig.add_subplot(1,1,1)
mulreg.scatter(p, y, color='g', alpha=0.1)
mulreg.set_title('Множественная линейная регрессия (ElasticNet)')

In [ ]:
mlpReg = MLPRegressor(hidden_layer_sizes=(600, ), activation='tanh', 
                      solver='adam', alpha=0.0001, batch_size='auto', 
                      learning_rate='adaptive', learning_rate_init=0.001, 
                      power_t=0.5, max_iter=10000, shuffle=True, random_state=None, 
                      tol=0.0001, verbose=False, warm_start=False, momentum=0.9, 
                      nesterovs_momentum=True, early_stopping=False, 
                      validation_fraction=0.1, beta_1=0.9, beta_2=0.999, 
                      epsilon=1e-08)

mlpReg = mlpReg.fit(X_train, y_train)
y_pred = mlpReg.predict(X_test)
mse_mlp = mean_squared_error(y_test, y_pred)
r2_mlp = r2_score(y_pred, y_test) 
r2_mlp = mlpReg.score(X_test, y_test)

print("\nAccuracy: %0.2f (+/- %0.2f)" % (np.mean(mse_mlp), np.std(mse_mlp) * 2))
print("Mean R^2: %0.2f\n" % (np.mean(r2_mlp))) 
    
fig=plt.figure(figsize=(7, 5))
mulreg=fig.add_subplot(1,1,1)
mulreg.scatter(p, y, color='g', alpha=0.1)
mulreg.set_title('MLP регрессия')  